In [2]:
#importing standard dependencies
import cv2 
import numpy as np 
import matplotlib.pyplot as plt
import os
import random


In [3]:
#import tensor flow dependencies fuctional api comps
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Conv2D, Dense, MaxPooling2D, Flatten, Input
import tensorflow as tf


In [4]:
ANC_PATH = r'C:\Users\CC\Documents\docs\Projects\Faceid\data\anchor'
POS_PATH = r'C:\Users\CC\Documents\docs\Projects\Faceid\data\positive'
NEG_PATH = r'C:\Users\CC\Documents\docs\Projects\Faceid\data\negative'
os.path.exists(ANC_PATH)

True

In [5]:
anchor = tf.data.Dataset.list_files(ANC_PATH + r'/*.jpg').take(300)
positive = tf.data.Dataset.list_files(POS_PATH + r'/*.jpg').take(300)
negative = tf.data.Dataset.list_files(NEG_PATH + r'/*.jpg').take(300)

In [6]:
def preprocess(file_path):
# read img from file (passes file path to the func)
  byte_img = tf.io.read_file(file_path)
# load the img  
  img = tf.io.decode_jpeg(byte_img)
# preprocess the img by resizing it into 100x100 pixels and 3 channels
  img = tf.image.resize(img, (100,100))
# scale img to be between 0 and 1
  img = img / 255.0
  return img

In [7]:
# outputs an array of ones which means maximum match( we want our model to output 1) which has
positives = tf.data.Dataset.zip((anchor, positive, tf.data.Dataset.from_tensor_slices(tf.ones(len(anchor)))))
# outputs 0 in the shape of our anchor and 
negatives = tf.data.Dataset.zip((anchor, negative, tf.data.Dataset.from_tensor_slices(tf.zeros(len(anchor)))))
#combines positives and negatives into a single dataset
data = positives.concatenate(negatives)

In [8]:
#preprocess positive and negative img input img = anchor validation_img = pos or neg
def preprocess_twin(input_img, validation_img, label):
  return (preprocess(input_img),preprocess(validation_img), label)

In [9]:
# building data loaderpipeline
data = data.map(preprocess_twin)
data = data.cache()
data = data.shuffle(buffer_size=1024)

In [10]:
# training partitions

# taking the length of our data and multiplying by .7 and rounding off and taking those images
train_data = data.take(round(len(data)*.7))
#traing data batches of 16 images
train_data = train_data.batch(16)
# starts preprocessing next set of images
train_data = train_data.prefetch(8)

In [11]:
# testing partitions

# skips the first 70% of the intial batch
test_data= data.skip(round(len(data)*.7))
# takes the next 30% of the data
test_data= test_data.take(round(len(data)*.3))
test_data = test_data.batch(16)
test_data = test_data.prefetch(8)

In [12]:
# Building Siamese model
# building embedding layer

def make_embedding():
# input layer with the shape (shape(100,100,3)
  inp = Input(shape=(100,100,3), name = 'input_image')
 
  # making the first block
  # First convolutional layer takes 64 filters on a pixel grid of 10x10 with activat= relu performed on inp
  c1 = Conv2D(64, (10,10),activation = 'relu')(inp)
  # performing max pooling on 2x2 grid
  m1 = MaxPooling2D(64, (2,2), padding = 'same')(c1)

  # making second block
  c2 = Conv2D(128,(7,7), activation = 'relu')(m1)
  m2 = MaxPooling2D(64,(2,2), padding = 'same')(c2)

  # third block
  c3 = Conv2D(128, (4,4), activation = 'relu')(m2)
  m3 = MaxPooling2D(64,(2,2), padding = 'same')(c3)

  # final embedding block
  c4 = Conv2D(256, (4,4), activation = 'relu')(m3)
  # flattening the outputs of c4 into a single dimen
  f1 = Flatten()(c4)

  d1 = Dense(4096, activation = 'sigmoid')(f1)

  return Model(inputs=[inp] ,outputs=[d1] ,name='embedding') 

In [13]:
embedding = make_embedding()

In [14]:
embedding.summary()

Model: "embedding"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_image (InputLayer)    [(None, 100, 100, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 91, 91, 64)        19264     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 46, 46, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 40, 40, 128)       401536    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 20, 20, 128)      0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 17, 17, 128)       26

In [15]:
# building siamese distance layer which compares the streams of images 

# L distance layer 
# inheritance
class L1Dist(Layer):
  def __init__(self , **kwargs):
    super().__init__()

  # important stuff- similarity calculation
  def call(self, input_embedding, validation_embedding):
    return tf.math.abs(input_embedding - validation_embedding)

In [16]:
l1 = L1Dist()

In [17]:
# handle inputs
def make_siamese_model():

  # anchor image input in the network
    input_image = Input(name= 'input_image', shape = (100,100,3))
  # validation image input in the network
    validation_image = Input(name='validation_image', shape=(100,100,3))

  # combining siamese distance components

    siamese_layer = L1Dist()
    siamese_layer._name = 'distance_layer'
    distances = siamese_layer(embedding(input_image), embedding(validation_image))

  # Classification layer for final output
  # gives 1 output
    classifier = Dense(1,activation = 'sigmoid')(distances)

    return Model(inputs = [input_image, validation_image], outputs=classifier, name='Siamese_Network')

In [18]:
siamese_model= make_siamese_model()

In [19]:
# setup loss and optimizers

# loss function estimates the error for our model
binary_cross_loss = tf.losses.BinaryCrossentropy()


In [20]:
# optimizer

opt = tf.keras.optimizers.Adam(1e-4) # learning rate for the opptimizer is 0.0001

In [21]:
# Build and train Step Fuction 

# will train a single batch
@tf.function
def train_step(batch):

# record and improve model
  with tf.GradientTape() as tape:
    # Get anchor and positive/negative image
    X = batch[:2]
    # Get label
    Y = batch[2]

    # forward pass
    ypredict = siamese_model(X, training = True)
    # Calculate Loss
    loss = binary_cross_loss(Y, ypredict)
    print(loss)

    # Calculate Gradients
    grad = tape.gradient(loss, siamese_model.trainable_variables)

    # Calculate updated waits and apply to siamese model
    opt.apply_gradients(zip(grad, siamese_model.trainable_variables))

    return loss

In [22]:
# building training Loop

def train(data, EPOCHS):
  #Loop through epochs
  for epoch in range(1, EPOCHS+1):
    print(' \n Epoch {}/{}'.format(epoch,EPOCHS))
    # for making a progress bar
    progbar = tf.keras.utils.Progbar(len(data))

    # loop through each batch
    # gives counter and batch
    for idx, batch in enumerate(data):
      train_step(batch)
      progbar.update(idx+1)

    # saving after every 10 epochs
    if epoch % 10==0:
      checkpoint.save(file_prefix=checkpoint_prefix)


In [24]:
# training the model
EPOCHS = 50

In [25]:
# Evaluating the model

# import metrics calculation
# precision demonstrates what proportion of positive identification was correct
# recall demonstrates what proportion of actual positives are correct

from tensorflow.keras.metrics import Precision, Recall
 

In [26]:
# reload the model

model = tf.keras.models.load_model(r'C:\Users\CC\Documents\docs\Projects\Faceid\siamesemodel.h5', custom_objects={'L1Dist':L1Dist, 'BinaryCrossentropy':tf.losses.BinaryCrossentropy})

In [27]:
# get a batch of test data
# gets data and converts it into numpy equiv
test_input,test_val, y_true = test_data.as_numpy_iterator().next()

In [28]:
# make predictions
ypredict = siamese_model.predict([test_input, test_val])
ypredict

array([[0.49988353],
       [0.5002953 ],
       [0.5021178 ],
       [0.5031449 ],
       [0.5003373 ],
       [0.5026527 ],
       [0.5013504 ],
       [0.501009  ],
       [0.5004959 ],
       [0.5040551 ],
       [0.50129944],
       [0.5035554 ],
       [0.5001931 ],
       [0.5020651 ],
       [0.50248414],
       [0.4999699 ]], dtype=float32)

In [29]:
# creating a metric object for correction of weights
m = Recall()
# Calculating the recall value 
m.update_state(y_true, ypredict)
# Return Recall Result
m.result().numpy()

0.75

In [30]:
# verification threshold = proportion of positive prediction / True positives
def verify( model, detection_threshold, verification_threshold):
# build results array
  results = []
# looping through the verication images folder
  for image in os.listdir(r'C:\Users\CC\Documents\Projects\Faceid\application_data\verification_images'):
    #using preprocess function on input and val image
    input_img = preprocess(r'C:\Users\CC\Documents\Projects\Faceid\application_data\input_image\input_image.jpg')
    validation_img = preprocess(os.path.join(r'C:\Users\CC\Documents\Projects\Faceid\application_data\verification_images',image))

    # make predictions
    # puts an array inside an array
    # puts all results inside result array
    result = model.predict(list(np.expand_dims([input_img,validation_img], axis = 1)))
    results.append(result)

# detection threshold = metrics above which a prediction is considered positive
  detection = np.sum(np.array(results) > detection_threshold)
# verification threshold = proportion of positive prediction / True positives
  verification = detection / len(os.listdir(r'C:\Users\CC\Documents\docs\Projects\Faceid\application_data\verification_images'))
  verified = verification > verification_threshold
  return results, verified

In [52]:


cap = cv2.VideoCapture(0)
while cap.isOpened():
    
    ret, frame = cap.read()
# cut down frame into 250 by 250 pixels
    #frame = frame[:250,:250,:]
    
    cv2.imshow('Verification', frame)
    
# verification trigger
    if cv2.waitKey(10) & 0XFF == ord('v'):
     # save input image to input image folder
        cv2.imwrite(r'C:\Users\CC\Documents\docs\Projects\Faceid\application_data\input_image\test.jpg', frame)
     # run verification
        results, verified = verify(model, 0.5, 0.5)
        print(verified)
    
    
    
# releases the webcam and destroys all windows
    if cv2.waitKey(1) & 0XFF == ord('q'):
        cap.release()
        cv2.destroyAllWindows()

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'C:\\Users\\CC\\Documents\\Projects\\Faceid\\application_data\\verification_images'

In [50]:
verified


NameError: name 'verified' is not defined

In [51]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    cv2.imshow('Verification', frame)
    if cv2.waitKey(1) & 0XFF == ord('q'):
        cap.release()
cv2.destroyAllWindows()